<a href="https://colab.research.google.com/github/DaniloLustosa-eng/Projeto-BigData/blob/main/big_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q pyspark

In [ ]:
import kagglehub

path_1 = kagglehub.dataset_download("ayushijain12345/tweeter")
path_2 = kagglehub.dataset_download("augustop/portuguese-tweets-for-sentiment-analysis")
path_3= kagglehub.dataset_download("derrickmwiti/24-thousand-tweets-later")


100%|██████████| 1.90M/1.90M [00:00<00:00, 67.0MB/s]

Extracting files...


100%|██████████| 224M/224M [00:01<00:00, 137MB/s]

Extracting files...


100%|██████████| 3.53M/3.53M [00:00<00:00, 128MB/s]

Extracting files...


In [ ]:
!sudo apt update
!sudo apt install openjdk-11-jdk -y

Hit:1 https://cli.github.com/packages stable InRelease
Get:2 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,434 kB]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,825 kB]
Get:14

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["PATH"] += ":/usr/lib/jvm/java-11-openjdk-amd64/bin"

In [ ]:
!java -version

openjdk version "11.0.28" 2025-07-15
OpenJDK Runtime Environment (build 11.0.28+6-post-Ubuntu-1ubuntu122.04.1)
OpenJDK 64-Bit Server VM (build 11.0.28+6-post-Ubuntu-1ubuntu122.04.1, mixed mode, sharing)


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Spark-1").getOrCreate()

In [ ]:
df_1= spark.read.csv(path_1, header=True, inferSchema=True)
df_2= spark.read.csv(path_2, header=True, inferSchema=True)
df_3= spark.read.csv(path_3, header=True, inferSchema=True)

In [ ]:
from pyspark.sql.functions import lower, col
df_normalizado_1 = df_1.withColumn("tweet", lower(col("tweet")))
df_normalizado_2 = df_2.withColumn("tweet_text", lower(col("tweet_text")))
df_normalizado_3 = df_3.withColumn("tweet", lower(col("tweet ")))

In [ ]:

df_filtrado_1= df_normalizado_1.filter(col("tweet").rlike(r"\b(java|python|javascript)\b"))
df_filtrado_2= df_normalizado_2.filter(col("tweet_text").rlike(r"\b(java|python|javascript)\b"))
df_filtrado_3= df_normalizado_3.filter(col("tweet").rlike(r"\b(java|python|javascript)\b"))

In [ ]:
df_filtrado_1.show(truncate = False)
df_filtrado_2.show(truncate = False)
df_filtrado_3.show(truncate = False)

+-----+-----+-------------------------------------------------------------------------------------------------------------------------------------+
|id   |label|tweet                                                                                                                                |
+-----+-----+-------------------------------------------------------------------------------------------------------------------------------------+
|898  |0    |python27 and concurrency are not best friends,   all the code i have 2 restructure to get concurrency with celery #developers #python|
|9505 |0    | @user #disneygatorattack 'no swimming' signs at gator dwelling lake.  is like: 'stay off grass' signs at the edge of a python iâ¦  |
|28723|0    |yay, passpo interview was successful, python will be mobile worldwide soon!   #inatearoomrightnow ;)                                 |
|31669|0    |and the dev grind begins again #webdev #html #css #java #work #late   #webdesign                   

In [ ]:
colunas = df_filtrado_3.columns

# Remove a sexta coluna
colunas_sem_sexta = [c for i, c in enumerate(colunas) if i != 5]

# Cria um novo DataFrame sem a sexta coluna
df_definitivos_3 = df_filtrado_3.select(*colunas_sem_sexta)


In [ ]:
#data-frames normalizados e com uma unica coluna
df_final_1 = df_filtrado_1.select("tweet")
df_final_2_errado = df_filtrado_2.select("tweet_text")
df_final_2 = df_final_2_errado.withColumnRenamed("tweet_text", "tweet")
df_final_3 = df_filtrado_3.select("tweet")
df_final_3.show(truncate = False)
df_final_2.show(truncate = False)
df_final_1.show(truncate = False)

+------------------------------------------------------------------------------------------------------------------------------------------------+
|tweet                                                                                                                                           |
+------------------------------------------------------------------------------------------------------------------------------------------------+
|rt @eugen_bleck: hack tool built with python @playwithpython.                                                                                   |
|rt @eugen_bleck: building a twitter bot at play with python meetup with @kevintabot.@activspaces @siliconmountain @gdgbuea #playwithpython…     |
|our most capable judges @monicamusonda of java foods, @jitokayumba of kukula capital &amp; dayu li of genesis energy de… https://t.co/b5jq0cvqk3|
|rt @relearnng: have you registered for the gap year club? give your teenagers lessons in python, html, and css throug

In [ ]:
df_final_unido = df_final_1.union(df_final_2).union(df_final_3)
df_final_unido.show(truncate = False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|tweet                                                                                                                                                                                                                                                                                      |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|python27 and concurrency are not best friends,   all the code i have 2 restructure to get concurrency with celery #developers #python        

In [ ]:
df_final_unido.coalesce(1) \
    .write \
    .mode("overwrite") \
    .option("header", True) \
    .csv("df_final_unido.csv")
